### Protocolos Clínicos de ARIA
# Procesamiento de prescripciones
------
**Objeto**: Extraer los datos de las prescripciones de pacientes para generar protocolos clínicos

Importación de módulos

In [1]:
import pandas as pd
import io
import re

Cambiar el directorio de trabajo, asumimos que el cuaderno está en el directorio `tools` dentro de la carpeta del proyecto

In [2]:
%cd ..

/Users/maqui/Documents/devel/rtpros


Leer los datos de una prescripción exportada de ARIA en un DataFrame de Pandas

In [3]:
prdf = pd.read_csv('prescripciones/orl_radical.csv')

## Órganos de riesgo

Los campos relativos a los órganos de riesgo están contenidos en el campo `OrganAtRisk`

Los separamos por líneas. La información contenida en cada línea no sigue un patrón regular

In [4]:
oar_lines = prdf.OrgansAtRisk.values[0].split('\n')

Definir un diccionario de expresión regulares para identificar la línea que encabeza la información de un órgano concreto y filtrar el nombre del órgano, la dosis media y la dosis máxima

In [79]:
rx_dict = {
    'Organ': re.compile(r'Organ :(?P<Organ>.*) Mean'),
    'Dmean': re.compile(r'Mean :(?P<Dmean>.*) Max Dose'),
    'Dmax' : re.compile(r'Max Dose :(?P<Dmax>.*)$'),
}

Definir una función que identifica el encabezado de cada órgano

In [80]:
def _parse_line(line):
    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """

    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None

Definir una función que procesando la línea de encabezado de un órgano genera un diccionario con el nombre del órgano, y los objetivos de dosis media y dosis máxima

In [81]:
def _parse_organ(line):
    """
    Do a regex search against all defined regexes and
    return a dictionary the key and match result

    """

    matches = {}
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            matches[key] = match.group(key)
        
    return matches

Procesar los órganos de riesgo generando una lista de sublistas.

Cada sublista commienza con la línea de encabezado de cada órgano. A continuación contiene una línea irrelevante con la indicación 'Constrains:' y depués una línea con cada restricción dosimétrica

In [82]:
oars, oar = [], None
for oar_line in oar_lines:
    oar_key, oar_name = _parse_line(oar_line)
    if oar_key:
        oars.append(oar)
        oar = [oar_line]
    else:
        oar.append(oar_line)
oars.remove(None)
oars

[['Organ :parótida derecha Mean : 26 Gy Max Dose : ',
  'Constraints : ',
  'V30$50 %'],
 ['Organ :parótida izda Mean : 26 Gy Max Dose : ',
  'Constraints : ',
  'V30$50 %'],
 ['Organ :médula Mean :  Max Dose : 45 Gy', 'Constraints : ', ''],
 ['Organ :tronco cerebral Mean :  Max Dose : ',
  'Constraints : ',
  'D53$100%',
  'V60$30%'],
 ['Organ :cristalino derecho Mean :  Max Dose : 8 Gy', 'Constraints : ', ''],
 ['Organ :cristalino izquierdo Mean :  Max Dose : 8 Gy', 'Constraints : ', ''],
 ['Organ :oido interno derecho Mean :  Max Dose : 50 Gy',
  'Constraints : ',
  ''],
 ['Organ :oido interno izquierdo Mean :  Max Dose : 50 Gy',
  'Constraints : ',
  ''],
 ['Organ :laringe Mean :  Max Dose : 70 Gy', 'Constraints : ', ''],
 ['Organ :mandíbula Mean :  Max Dose : 70 Gy', 'Constraints : ', ''],
 ['Organ :tiroides Mean :  Max Dose : 45 Gy', 'Constraints : ', ''],
 ['Organ :quiasma óptico Mean :  Max Dose : 50 Gy', 'Constraints : ', ''],
 ['Organ :n óptico derecho Mean :  Max Dose : 50 G

In [100]:
oars_list = []
for oar in oars:
    oar_dict = _parse_organ(oar[0])
    oar_dict['DosimPars'] = oar[2:]
    oars_list.append(oar_dict)
oardf = pd.DataFrame(oars_list)
oardf

,Organ,Dmean,Dmax,DosimPars
0,parótida derecha,26 Gy,,[V30$50 %]
1,parótida izda,26 Gy,,[V30$50 %]
2,médula,,45 Gy,[]
3,tronco cerebral,,,"[D53$100%, V60$30%]"
4,cristalino derecho,,8 Gy,[]
5,cristalino izquierdo,,8 Gy,[]
6,oido interno derecho,,50 Gy,[]
7,oido interno izquierdo,,50 Gy,[]
8,laringe,,70 Gy,[]
9,mandíbula,,70 Gy,[]


In [101]:
for dpar in oardf[oardf.Organ == 'tronco cerebral'].DosimPars.values[0]:
    print(dpar)

D53$100%
V60$30%
